In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# Plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# ML
from soap import extract_species_pair_groups

from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.multiclass import OneVsRestClassifier

# Utilities
import h5py
import json
import itertools
from tqdm.auto import tqdm
import project_utils as utils
from tools import load_json, save_json
from tempfile import mkdtemp
from shutil import rmtree

# Import COSMO style toolkit
import cosmoplot.colorbars as cosmocbars
import cosmoplot.utils as cosmoutils
import cosmoplot.style as cosmostyle

cosmostyle.set_style('article')
colorList = cosmostyle.color_cycle

# Load train and test splits

In [3]:
# Load SOAP cutoffs
soap_hyperparameters = load_json('../Processed_Data/soap_hyperparameters.json')   
cutoffs = soap_hyperparameters['interaction_cutoff']

In [4]:
# Load train sets for IZA and Deem
iza_train_idxs = np.loadtxt('../Processed_Data/IZA_230/svm_train.idxs', dtype=int)
iza_sort_idxs = np.argsort(iza_train_idxs)
iza_unsort_idxs = np.argsort(iza_sort_idxs)
deem_train_idxs = np.loadtxt('../Processed_Data/DEEM_330k/svm_train.idxs', dtype=int)

In [5]:
# Load cantons for IZA and Deem
iza_cantons = np.loadtxt('../Raw_Data/IZA_230/cantons.dat', usecols=1, dtype=int)
deem_cantons_2 = np.loadtxt('../Processed_Data/DEEM_330k/Data/cantons_2-class.dat', dtype=int)
deem_cantons_4 = np.loadtxt('../Processed_Data/DEEM_330k/Data/cantons_4-class.dat', dtype=int)
n_deem = len(deem_cantons_2)

In [6]:
# Build set of "master" canton labels
cantons = {}

cantons[4] = np.concatenate((
    iza_cantons[iza_train_idxs], 
    deem_cantons_4[deem_train_idxs]
))

cantons[2] = np.concatenate((
    np.ones(len(iza_train_idxs), dtype=int),
    deem_cantons_2[deem_train_idxs]
))

# Build set of class weights (by sample) for centering and scaling
class_weights = {n_cantons: utils.balanced_class_weights(cantons[n_cantons]) for n_cantons in (2, 4)}

In [7]:
# Load dummy Deem cantons to test the "null" case
dummy_cantons = {}
dummy_cantons[2] = np.loadtxt('../Processed_Data/DEEM_330k/Data/dummy_cantons_2-class.dat', dtype=int)
dummy_cantons[2] = dummy_cantons[2][deem_train_idxs]
dummy_cantons[4] = np.loadtxt('../Processed_Data/DEEM_330k/Data/dummy_cantons_4-class.dat', dtype=int)
dummy_cantons[4] = dummy_cantons[4][deem_train_idxs]

# Build set of dummy class weights (by sample) for centering and scaling
dummy_class_weights = {n_cantons: utils.balanced_class_weights(dummy_cantons[n_cantons]) for n_cantons in (2, 4)}

# Model setup

In [8]:
model_dir = '../Processed_Data/Models'

deem_name = 'DEEM_330k'
iza_name = 'IZA_230'
deem_dir = f'../Processed_Data/{deem_name}/Data'
iza_dir = f'../Processed_Data/{iza_name}/Data'

# Linear SVC

In [9]:
batch_size = 20000

In [10]:
# Linear model setup
n_species = 2
group_names = {
    'power': ['OO', 'OSi', 'SiSi', 'OO+OSi', 'OO+SiSi', 'OSi+SiSi', 'OO+OSi+SiSi'], 
    'radial': ['O', 'Si', 'O+Si']
}

In [11]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    linear_dir = f'{model_dir}/{cutoff}/LSVC'
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_soaps = utils.load_hdf5(iza_file)
        
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        f = h5py.File(deem_file, 'r')
        deem_330k_dataset = f['0']
        
        soaps = np.vstack((iza_soaps[iza_train_idxs], deem_330k_dataset[deem_train_idxs]))

        n_features = soaps.shape[1]
        feature_groups = extract_species_pair_groups(
            n_features, n_species, 
            spectrum_type=spectrum_type,
            combinations=True
        )
        
        # Prepare batches for SVM
        n_samples_330k = deem_330k_dataset.len()
        n_batches = n_samples_330k // batch_size
        if n_samples_330k % batch_size > 0:
            n_batches += 1
        
        for species_pairing, feature_idxs in zip(
            tqdm(group_names[spectrum_type], desc='Species', leave=False),
            feature_groups
        ):
            
            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
                
                # Prepare outputs
                output_dir = f'LSVC/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                
                os.makedirs(f'{deem_dir}/{cutoff}/{output_dir}', exist_ok=True)
                os.makedirs(f'{iza_dir}/{cutoff}/{output_dir}', exist_ok=True)
                
                parameter_dir = f'{linear_dir}/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                
                svc_parameters = load_json(f'{parameter_dir}/svc_parameters_balanced_accuracy.json')
                
                # IZA+DEEM classification
                pipeline = Pipeline(
                    [
                        ('norm_scaler', utils.StandardNormScaler()),
                        ('kernel_constructor', utils.KernelConstructor()),
                        ('svc', OneVsRestClassifier(SVC(**svc_parameters), n_jobs=4))
                    ],
                )
                fit_params = {'norm_scaler__sample_weight': class_weights[n_cantons]}
                pipeline.fit(soaps[:, feature_idxs], cantons[n_cantons], **fit_params)
                
                # Read the IZA structures and compute decision functions
                # and canton predictions
                iza_dfs = pipeline.decision_function(iza_soaps[:, feature_idxs])
                iza_predicted_cantons = pipeline.predict(iza_soaps[:, feature_idxs])
                
                np.savetxt(f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat', iza_dfs)
                np.savetxt(f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_cantons.dat', iza_predicted_cantons, fmt='%d')
                
                # Read the DEEM structures and compute decision functions
                # and canton predictions
                if n_cantons == 2:
                    deem_dfs = np.zeros(n_deem)
                else:
                    deem_dfs = np.zeros((n_deem, n_cantons))
                
                deem_predicted_cantons = np.zeros(n_deem)
                
                for i in tqdm(range(0, n_batches), desc='Batch', leave=False):
                    batch_slice = slice(i * batch_size, (i + 1) * batch_size)
            
                    deem_330k_batch = deem_330k_dataset[batch_slice, feature_idxs]
                    deem_dfs[batch_slice] = pipeline.decision_function(deem_330k_batch)
                    deem_predicted_cantons[batch_slice] = pipeline.predict(deem_330k_batch)
                
                np.savetxt(f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat', deem_dfs)
                np.savetxt(f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_cantons.dat', deem_predicted_cantons, fmt='%d')
                
                # Save the SVC model and the scaler
                # We don't save the KernelConstructor b/c it is really big
                save_json(pipeline.named_steps['norm_scaler'].__dict__, f'{parameter_dir}/norm_scaler.json', array_convert=True)
                save_json(pipeline.named_steps['svc'].__dict__, f'{parameter_dir}/svc.json', array_convert=True)

        f.close()

# Linear SVC for dummy models

In [12]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    linear_dir = f'{model_dir}/{cutoff}/LSVC'
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        f = h5py.File(deem_file, 'r')
        deem_330k_dataset = f['0']
        
        deem_soaps = deem_330k_dataset[deem_train_idxs]
        
        n_features = len(deem_330k_dataset[0, :])
        feature_groups = extract_species_pair_groups(
            n_features, n_species, 
            spectrum_type=spectrum_type,
            combinations=True
        )
        
        # Prepare batches for SVM
        n_samples_330k = deem_330k_dataset.len()
        n_batches = n_samples_330k // batch_size
        if n_samples_330k % batch_size > 0:
            n_batches += 1
        
        for species_pairing, feature_idxs in zip(
            tqdm(group_names[spectrum_type], desc='Species', leave=False),
            feature_groups
        ):
            
            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
                
                # Prepare outputs
                output_dir = f'LSVC/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                
                os.makedirs(f'{deem_dir}/{cutoff}/{output_dir}', exist_ok=True)
                
                parameter_dir = f'{linear_dir}/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                
                dummy_svc_parameters = load_json(f'{parameter_dir}/dummy_svc_parameters_balanced_accuracy.json')
                
                # Dummy DEEM classification
                dummy_pipeline = Pipeline(
                    [
                        ('norm_scaler', utils.StandardNormScaler()),
                        ('kernel_constructor', utils.KernelConstructor()),
                        ('svc', OneVsRestClassifier(SVC(**dummy_svc_parameters), n_jobs=4))
                    ],
                )
                dummy_fit_params = {'norm_scaler__sample_weight': dummy_class_weights[n_cantons]}
                dummy_pipeline.fit(deem_soaps[:, feature_idxs], dummy_cantons[n_cantons], **dummy_fit_params)
                
                if n_cantons == 2:
                    dummy_dfs = np.zeros(n_deem)
                else:
                    dummy_dfs = np.zeros((n_deem, n_cantons))
                
                dummy_predicted_cantons = np.zeros(n_deem)
                
                for i in tqdm(range(0, n_batches), desc='Batch', leave=False):
                    batch_slice = slice(i * batch_size, (i + 1) * batch_size)
            
                    deem_330k_batch = deem_330k_dataset[batch_slice, feature_idxs]
                    dummy_dfs[batch_slice] = dummy_pipeline.decision_function(deem_330k_batch)
                    dummy_predicted_cantons[batch_slice] = dummy_pipeline.predict(deem_330k_batch)
 
                np.savetxt(f'{deem_dir}/{cutoff}/{output_dir}/dummy_svc_structure_dfs.dat', dummy_dfs)
                np.savetxt(f'{deem_dir}/{cutoff}/{output_dir}/dummy_svc_structure_cantons.dat', dummy_predicted_cantons, fmt='%d')
                
                # Save the dummy SVC model and the scaler
                # We don't save the KernelConstructor b/c it is really big
                save_json(dummy_pipeline.named_steps['norm_scaler'].__dict__, f'{parameter_dir}/dummy_norm_scaler.json', array_convert=True)
                save_json(dummy_pipeline.named_steps['svc'].__dict__, f'{parameter_dir}/dummy_svc.json', array_convert=True)
                                
        f.close()